### Step 1: Read in data and drop all data for non-K12 persons 
Probably will have to do this for all analyses, unless we are looking also at family members. The data needs to include the "GRADEATT" and "SCHOOL" attributes. This portion starts with a df and creates df_k12 which includes only K-12 persons. 

See internet_access.ipynb for details on steps/the most comprehensive documentation out of all of these. 

In [1]:
import csv
import json
import numpy as np
import pandas as pd

df = pd.read_csv("../../../acs_data.csv")
df_students = df[df['SCHOOL'] == 2]
df_k12 = df_students[(df_students.GRADEATT != 1) & (df_students['GRADEATT'] != 6) & (df_students['GRADEATT'] != 7)]

## Bedrooms

In [43]:
# Drop rows w/o info on internet access
df_k12 = df_k12[df_k12['BEDROOMS'] != 0]

In [44]:
df_k12['few_rooms'] = 0
df_k12.loc[df_k12['BEDROOMS'] < df_k12['NUMPREC'] - 1,'few_rooms'] = 1

In [45]:
sum(df_k12['few_rooms'])/len(df_k12['few_rooms'])

0.1727517020101407

In [46]:
df_k12['few_rooms'] = df_k12['few_rooms'] * df_k12['PERWT']

In [47]:
sum(df_k12['PERWT'])

53592256.0

In [49]:
# percentage of students overall living in non-ideal room-number situations
sum(df_k12['few_rooms'])/sum(df_k12['PERWT'])

0.18298878852944725

In [27]:
# Get conversion from fips to state
with open("../FIPSconversion/fipsToState.json") as json_file:
    state_fips = json.load(json_file)

In [33]:
l = [] # list of dictionaries

for fips in state_fips:
    few_rooms = df_k12.loc[df_k12['STATEFIP'] == int(fips), 'few_rooms'].sum()
    total = df_k12.loc[df_k12['STATEFIP'] == int(fips), 'PERWT'].sum()
    rate = int((few_rooms / total) * 100)
    state_name = state_fips[fips]
    l.append({'AREANM':state_name, 'AREACD':state_name, 'value':rate})


In [ ]:
# output as CSV
out = pd.DataFrame(l)
out.to_csv('../bedrooms.csv')

## Computer access

In [36]:
# define feature-specific variables
col = 'CILAPTOP'
new_col = 'no_comp'

In [37]:
# Drop rows w/o info on desired feature
df_k12 = df_students[(df_students.GRADEATT != 1) & (df_students['GRADEATT'] != 6) & (df_students['GRADEATT'] != 7)]
df_k12 = df_k12[df_k12[col] != 0]

In [40]:
df_k12[new_col] = 0
df_k12.loc[df_k12[col] == 2,new_col] = 1

In [24]:
df_k12[new_col] = df_k12[new_col] * df_k12['PERWT']

In [50]:
sum(df_k12[new_col])/sum(df_k12['PERWT'])

0.0012056592653983441

In [51]:
sum(df_k12[new_col])

64614

In [56]:
l = [] # list of dictionaries

for fips in state_fips:
    barriered = df_k12.loc[df_k12['STATEFIP'] == int(fips), new_col].sum()
    total = df_k12.loc[df_k12['STATEFIP'] == int(fips), 'PERWT'].sum()
    rate = (barriered / total) * 100
    #rate = int((barriered / total) * 100)
    state_name = state_fips[fips]
    l.append({'AREANM':state_name, 'AREACD':state_name, 'value':barriered})

# output as CSV
out = pd.DataFrame(l)
out.to_csv('../'+new_col+'.csv', index = False)

## High speed internet

In [57]:
# define feature-specific variables
col = 'CIHISPEED'
new_col = 'no_hispeed'

In [63]:
# Drop rows w/o info on desired feature
df_k12 = df_students[(df_students.GRADEATT != 1) & (df_students['GRADEATT'] != 6) & (df_students['GRADEATT'] != 7)]
df_k12 = df_k12[df_k12[col] != 0]

In [64]:
df_k12[new_col] = 0
df_k12.loc[df_k12[col] > 18, new_col] = 1

In [65]:
df_k12[new_col] = df_k12[new_col] * df_k12['PERWT']

In [66]:
# Nation-wide rate
sum(df_k12[new_col])/sum(df_k12['PERWT'])

0.16729372865461759

In [67]:
# Nation-wide number
sum(df_k12[new_col])

8285868.0

In [68]:
l = [] # list of dictionaries

for fips in state_fips:
    barriered = df_k12.loc[df_k12['STATEFIP'] == int(fips), new_col].sum()
    total = df_k12.loc[df_k12['STATEFIP'] == int(fips), 'PERWT'].sum()
    rate = int((barriered / total) * 100)
    state_name = state_fips[fips]
    l.append({'AREANM':state_name, 'AREACD':state_name, 'value':rate})

# output as CSV
out = pd.DataFrame(l)
out.to_csv('../'+new_col+'.csv', index = False)